In [ ]:
# --- 1. 라이브러리 설치 및 임포트 ---

import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [ ]:
# --- 2. 경로 설정 및 데이터 로드 ---

# 학습 데이터 파일 경로
TRAIN_DATA_PATH = '../../data/processed/train_geocoded_sample.csv'
SUBWAY_DATA_PATH = '../../data/raw/subway_feature.csv'
BUS_DATA_PATH = '../../data/raw/bus_feature.csv'

# 출력 파일 경로 설정
OUTPUT_DIR = '../../data/processed/transportation-features/'
OUTPUT_FILENAME = 'train_transportation_features.csv'
OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

# 데이터 로드
print("데이터를 로드합니다...")

try:
    # pandas에서 dtype 관련 경고 제거를 위해 low_memory=False 설정
    train_df = pd.read_csv(TRAIN_DATA_PATH, low_memory=False)
    subway_df = pd.read_csv(SUBWAY_DATA_PATH)
    bus_df = pd.read_csv(BUS_DATA_PATH)
    
    print("모든 데이터 로드 완료.")
    
    data_loaded_successfully = True
    
except FileNotFoundError as e:
    print(f"오류: 파일 로드에 실패했습니다. 경로를 확인해주세요. \n>> {e}")
    data_loaded_successfully = False

In [ ]:
# --- 3. Haversine 거리 계산 함수 정의 ---
def haversine_distance_vectorized(lon1, lat1, lon2, lat2):
    R = 6371  # 지구 반지름 (단위: km)

    # 입력된 위도와 경도를 라디안 단위로 변환
    lon1_rad, lat1_rad, lon2_rad, lat2_rad = map(np.radians, [lon1, lat1, lon2, lat2])

    # 출발지 좌표를 세로 방향으로 reshape (벡터화 계산을 위한 준비)
    lon1_rad = lon1_rad.values[:, np.newaxis]
    lat1_rad = lat1_rad.values[:, np.newaxis]

    # 도착지 좌표와 출발지 좌표 간의 차이 계산
    dlon = lon2_rad - lon1_rad  # 경도 차이
    dlat = lat2_rad - lat1_rad  # 위도 차이

    # Haversine 공식의 중간 계산 단계
    a = (
        np.sin(dlat / 2.0)**2
        + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
    )

    # 거리의 중심각 계산 (라디안 기준)
    c = 2 * np.arcsin(np.sqrt(a))

    # 최종 거리 계산 (단위: km)
    distance = R * c

    return distance  # 반환값: 2차원 배열 (출발지 × 도착지 간 거리)

In [ ]:
# --- 4. 피처 생성 및 저장 (데이터 로드 성공 시에만 실행) ---
if data_loaded_successfully:
    try:
        # --- 지하철 관련 피처 생성 ---
        print("\n지하철 관련 피처 생성을 시작합니다...")

        # 지하철역까지의 거리 계산 (모든 아파트 × 지하철역 간 거리 행렬 생성)
        distance_matrix_subway = haversine_distance_vectorized(
            train_df['좌표X'], train_df['좌표Y'],  # 아파트 위치 (출발지)
            subway_df['경도'], subway_df['위도']   # 지하철 위치 (도착지)
        )

        # 각 아파트에서 가장 가까운 지하철역까지의 거리 (최소값)
        train_df['지하철최단거리'] = distance_matrix_subway.min(axis=1)

        # 반경 1km 이내에 존재하는 지하철역 개수
        train_df['반경_1km_내_지하철역_수'] = (distance_matrix_subway < 1).sum(axis=1)

        print("지하철 관련 피처 생성 완료.")

        # --- 버스 관련 피처 생성 ---
        print("\n버스 관련 피처 생성을 시작합니다...")

        # 버스정류장까지의 거리 계산 (모든 아파트 × 버스정류장 간 거리 행렬 생성)
        distance_matrix_bus = haversine_distance_vectorized(
            train_df['좌표X'], train_df['좌표Y'],  # 아파트 위치 (출발지)
            bus_df['X좌표'], bus_df['Y좌표']       # 버스정류장 위치 (도착지)
        )

        # 각 아파트에서 가장 가까운 버스정류장까지의 거리
        train_df['버스최단거리'] = distance_matrix_bus.min(axis=1)

        # 반경 500m 이내에 존재하는 버스정류장 개수
        train_df['반경_500m_내_버스정류장_수'] = (distance_matrix_bus < 0.5).sum(axis=1)

        print("버스 관련 피처 생성 완료.")

        # --- 결과 확인 및 저장 ---
        print("\n생성된 피처 미리보기:")
        print(train_df[[
            '지하철최단거리',
            '반경_1km_역수',
            '반경_500m_역수',
            '반경_300m_역수',
            '버스최단거리',
            '반경_1km_버스정류장_수',
            '반경_500m_버스정류장_수',
            '반경_300m_버스정류장_수',
        ]].head())

        # 결과 저장 디렉토리 생성 (이미 존재하면 무시)
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        # 최종 결과 CSV로 저장
        train_df.to_csv(OUTPUT_PATH, index=False)
        display(train_df)

        print(f"\n모든 작업이 완료되었습니다. 최종 데이터가 '{OUTPUT_PATH}' 경로에 저장되었습니다.")

    except Exception as e:
        # 예외 발생 시 에러 메시지 출력
        print(f"\n오류: 피처 생성 또는 파일 저장 중 문제가 발생했습니다. \n>> {e}")